In [1]:
import os
from config import dataset_dict
from shutil import copy
import json
import cv2
import numpy as np
from copy import deepcopy
from utils.data_augmentation import adjust_gamma, homography, horizontal_flip
from data.handset.dataset_function import validate_keypoints, validate_label, get_bbox
from tqdm import tqdm
# from utils.visualization_tools import draw_bbox, draw_point


def data_augmentation(image, keypoints, bbox, prob=0.5):
    """ data augmentation for specified image

    Args:
        image (np.ndarray): image read by cv2 
        keypoints (list): [x1, y1 ,v1, ... ,x21, y21, v21]
        bbox (list): [lx, ly, w, h]
        prob (float): the probability of data augmentation
    """
    H, W, C = image.shape
    if image is None:
            raise ValueError(' Fail to read %s' % image)
    keypoints = np.array(keypoints, dtype=np.float32).reshape((21, 3))
    bbox = np.array(bbox, dtype=np.float32)

    # get crop region
    x, y, w, h = bbox  # topleft_x, topleft_y, width, height
    scale_factor = np.random.random()
    dx = (scale_factor * w) // 2
    dy = (scale_factor * h) // 2
    
    x1 = max(0, int(x - dx))
    y1 = max(0, int(y - dy))
    x2 = min(W, int(x + w + dx)) 
    y2 = min(H, int(y + h + dy)) 
    
    h_crop, w_crop = y2 - y1, x2 - x1

    img_crop = image[y1:y2, x1:x2]
    image = cv2.resize(img_crop, (W, H), interpolation=cv2.INTER_NEAREST)
    
    scale_factor = W / w_crop, H / h_crop
    keypoints[:, :2] -= [x1, y1]
    keypoints[:, :2] *= scale_factor
    bbox[:2] -= [x1, y1]
    bbox[:2] *= scale_factor  
    bbox[2:] *= scale_factor  
    
    image = adjust_gamma(image, prob=0.5)
    image, keypoints, bbox = homography(image, keypoints[None],
                                        prob=prob, bbox=bbox[None],
                                        topleft=True)
    image, keypoints, bbox = horizontal_flip(image, keypoints,
                                                prob=prob, bbox=bbox)
    # 验证关键点是否可见，不可见设置为-1     
    keypoints = validate_keypoints(keypoints, img_size=[H, W])  
    keypoints = keypoints[0]  # (1, 21, 3) -> (21, 3) 
    bbox = bbox[0].tolist()
    
    # image = draw_point(image, keypoints)
    keypoints = keypoints.reshape((keypoints.size)).tolist()
   
    # bbox = bbox.tolist()
    # image = draw_bbox(image, bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3])
    return image, keypoints, bbox

In [2]:
def generate_freiahand_multiscale_dataset(save_root='./'):
    
    freihand = dataset_dict['freihand']
    freihand_root = freihand['root']
    test_file = freihand['test_file']
    train_file = freihand['train_file']
    val_file = freihand['val_file']
    
    new_ann_dir = os.path.join(save_root, "annotations")
    new_images_dir = os.path.join(save_root, "images")
    new_test_file = os.path.join(new_ann_dir, "freihand_test.json") 
    new_train_val_file = os.path.join(new_ann_dir, "freihand_train_val.json")

    config_info = dict(
        name="freihand_plus",
        root=save_root,
        test_file=new_test_file,      
        train_file=new_train_val_file,
    )

    if not os.path.exists(save_root):
        os.makedirs(save_root)
        print("mkdir save_root : {}".format(save_root))
    
    if not os.path.exists(new_ann_dir):
        os.mkdir(new_ann_dir)
        print("mkdir new_ann_path : {}".format(save_root))
     
    if not os.path.exists(new_images_dir):
        os.mkdir(new_images_dir)
        print("mkdir new_images_path : {}".format(save_root))
    
    config_save_path = os.path.join(save_root, "config_info.json")
    json.dump(config_info, open(config_save_path, 'w'), indent=4)
    

    test_dict = json.load(open(test_file, 'r'))
    # move test set 
    g_imgs = []
    for img_info in tqdm(test_dict['images'], desc='move test set '):
        # copy image files to new directory
        file_name = img_info['file_name']
        base_file_name = os.path.basename(file_name)
        
        image_path = os.path.join(freihand_root, img_info['file_name'])
        copy(
                image_path,
                os.path.join(new_images_dir, base_file_name) 
            )
        
        img_info['file_name'] = "images/" + base_file_name
        g_imgs.append(img_info)

    test_dict['images'] = g_imgs
    json.dump(test_dict, open(new_test_file, 'w'))


    # merge train_set and val_set
    g_imgs = []
    g_anns = []

    train_dict = json.load(open(train_file, 'r'))
    val_dict = json.load(open(val_file, 'r'))
    
    images_list = train_dict['images'] + val_dict['images']
    anns_list = train_dict['annotations'] + val_dict['annotations']
    
    assert len(images_list) == len(anns_list), \
        "{} <> {}".format(len(images_list), len(anns_list))
        
    max_id = max([img['id'] for img in images_list])
    total_new = 0
    for i  in tqdm(range(len(images_list)),
                   desc='merge train_set and val_set'):
        img_info = images_list[i]
        ann_info = anns_list[i]
        
        img_copy = deepcopy(img_info)
        ann_copy = deepcopy(ann_info)
        # copy image files to new directory
        file_name = img_copy['file_name']
        base_file_name = os.path.basename(file_name)
        
        image_path = os.path.join(freihand_root, img_copy['file_name'])
        copy(
                image_path,
                os.path.join(new_images_dir, base_file_name) 
            )
           
        img_copy['file_name'] = "images/" + base_file_name
        g_imgs.append(img_copy)
        g_anns.append(ann_copy)
        
        # generate mutiscale samples
        if np.random.random() < 0.5:
            total_new += 1
            
            img_copy = deepcopy(img_info)
            ann_copy = deepcopy(ann_info)
            
            image = cv2.imread(image_path)
            keypoints = ann_copy['keypoints']
            bbox = ann_copy['bbox']
            
            image, keypoints, bbox = \
                data_augmentation(image, keypoints, bbox, prob=0)
            
            new_file_name = "ms_" + base_file_name
            # record
            new_id = max_id + total_new
            img_copy['file_name'] = "images/" +  new_file_name
            img_copy['id'] = new_id
            
            ann_copy['keypoints'] = keypoints
            ann_copy['bbox'] = bbox
            ann_copy['id'] = new_id
            ann_copy['image_id'] = new_id
            
            save_img_path = os.path.join(new_images_dir, new_file_name)
            flag = cv2.imwrite(save_img_path, image)
            if not flag:
                raise ValueError("image save error!!!")

            g_imgs.append(img_copy)
            g_anns.append(ann_copy)
    
    train_dict['images'] = g_imgs 
    train_dict['annotations'] = g_anns
    json.dump(train_dict, open(new_train_val_file, 'w'))
    
    print("work done successfully")
    print("the number of test_set: \t\t{}".format(len(test_dict['images'])))
    print("the number of new samples: \t\t {}".format(total_new))
    print("the number of train_val_set: \t\t{}".format(len(g_anns)))


In [3]:
generate_freiahand_multiscale_dataset(
    save_root='/root/data/Dataset/freihand_plus')

mkdir save_root : /root/data/Dataset/freihand_plus
mkdir new_ann_path : /root/data/Dataset/freihand_plus
mkdir new_images_path : /root/data/Dataset/freihand_plus


merge train_set and val_set: 100%|██████████| 117216/117216 [18:10<00:00, 107.52it/s]


work done successfully
the number of test_set: 		13024
the number of new samples: 		 58507
the number of train_val_set: 		175723
